<a href="https://colab.research.google.com/github/Gaukhar-ai/for_my_Thinkful_work/blob/master/Copy_of_Time_Series_Analysis_Day_3_Afternoon_Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv('/content/drive/My Drive/DSI Month 4/Week 12/Datasets/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data[['Date', 'Close']]

FileNotFoundError: ignored

In [ ]:
def ilinechart(df, x, y, groups=None, title=''):
    fig = px.line(df, x=x, y=y, color=groups, title=title, 
                  template='none').update(layout=dict(title=dict(x=0.5)))
    
    fig.show()

In [ ]:
ilinechart(data, 'Date', 'Close')

In [ ]:
data['20_Day'] = data['Close'].rolling(window=20).mean()
data['50_Day'] = data['Close'].rolling(window=50).mean()
data['100_Day'] = data['Close'].rolling(window=100).mean()
data['200_Day'] = data['Close'].rolling(window=200).mean()


In [ ]:
melted = pd.melt(data, id_vars='Date', value_vars=['Close', '20_Day', '50_Day', '100_Day', '200_Day'], 
                 var_name='Variable', value_name='Value')

ilinechart(melted, 'Date', 'Value', groups='Variable')

In [ ]:
def wma(df, field, window):
    weights = np.arange(1, window + 1)
    ma = df[field].rolling(window)
    wma = ma.apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
    return wma

In [ ]:
data['200_Day_WMA'] = wma(data, 'Close', 200)

In [ ]:
melted = pd.melt(data, id_vars='Date', value_vars=['Close', '200_Day', '200_Day_WMA'], 
                 var_name='Variable', value_name='Value')

ilinechart(melted, 'Date', 'Value', groups='Variable')

In [ ]:
model = sm.tsa.ExponentialSmoothing(data['Close']).fit()
data['200_Day_SExp'] = model.predict(200)

In [ ]:
melted = pd.melt(data, id_vars='Date', value_vars=['Close', '200_Day', '200_Day_WMA', '200_Day_SExp'], 
                 var_name='Variable', value_name='Value')

ilinechart(melted, 'Date', 'Value', groups='Variable', title='Smoothing Method Comparison')

In [ ]:
model = sm.tsa.ExponentialSmoothing(data['Close'], trend='add').fit()
data['200_Day_DExp'] = model.predict(200)

model = sm.tsa.ExponentialSmoothing(data['Close'], trend='add', 
                                    seasonal='add', 
                                    seasonal_periods=4).fit()

data['200_Day_TExp'] = model.predict(200)

In [ ]:
melted = pd.melt(data, id_vars='Date', var_name='Variable', value_name='Value',
                 value_vars=['Close','200_Day_SExp', '200_Day_DExp', '200_Day_TExp'])

ilinechart(melted, 'Date', 'Value', groups='Variable', title='Smoothing Method Comparison')

In [ ]:
data['SExp_Diff'] = data['Close'] - data['200_Day_SExp']
data['DExp_Diff'] = data['Close'] - data['200_Day_DExp']
data['TExp_Diff'] = data['Close'] - data['200_Day_TExp']

print('Simple MAE: ', data['SExp_Diff'].abs().mean(), 
      'RMSE: ', np.sqrt(np.mean(data['SExp_Diff']**2)))

print('Double MAE:', data['DExp_Diff'].abs().mean(), 
      'RMSE:', np.sqrt(np.mean(data['DExp_Diff']**2)))

print('Triple MAE:', data['TExp_Diff'].abs().mean(), 
      'RMSE:', np.sqrt(np.mean(data['TExp_Diff']**2)))

Simple MAE:  1.6021316456279393 RMSE:  2.8064834855263032
Double MAE: 1.5974217186030153 RMSE: 2.8028819380999987
Triple MAE: 1.5978879188673496 RMSE: 2.8027250318279004
